# Week4 授業課題

___
## 問題２：学習と検証

### import section

In [15]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier

/usr/local/var/pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### read csv

In [2]:
train_dataset = pd.read_csv("../Week3/application_train.csv")
test_dataset = pd.read_csv("../Week3/application_test.csv")

# train_dataset.info()

### データの前処理

In [42]:
# Concatenate two dataset to treat as one
# train_test_df = pd.concat([train_dataset, test_dataset], axis=0, sort=False)

# Fill missings
train_dataset_filled = train_dataset.fillna(-1)
test_dataset_filled = test_dataset.fillna(-1)

# Replace categorical value to dummy value
for col in train_dataset_filled.columns:
    labels, uniques = pd.factorize(train_dataset_filled[col])
    train_dataset_filled[[col]] = labels

for col in test_dataset_filled.columns:
    labels, uniques = pd.factorize(test_dataset_filled[col])
    test_dataset_filled[[col]] = labels
                                     

### テスト用データの前処理

In [48]:
train_X = train_dataset_filled.drop(["TARGET"], axis=1)
train_Y = train_dataset_filled["TARGET"]

### トレーニング用とテスト用のデータ分割

In [49]:
# Create train data and test data from train_dataset_nomiss
x_train, x_test, y_train, y_test = train_test_split(np.array(train_X), \
                                                    np.array(train_Y), \
                                                    train_size=0.70, test_size=0.30)
                                                                                           

### ランダムフォレストのモデルによる評価（モデル作成が主目的）

In [50]:
clf = RandomForestClassifier(random_state=0)
clf = clf.fit(x_train, y_train)
pred = clf.predict(x_test)
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
auc(fpr, tpr)

0.5054595790296003

In [51]:
accuracy_score(pred, y_test)

0.9160253213952783

### 本番のテストデータで評価

In [53]:
pred = clf.predict_proba(test_dataset_filled)

### Kaggle提出用のフォーマットによるファイル出力

In [75]:
pred_reshape = pred[:, 0:1].reshape(len(pred))

In [86]:
commit_format = pd.DataFrame({"SK_ID_CURR" : test_dataset["SK_ID_CURR"],
                             "TARGET" : pred_reshape})

In [87]:
commit_format.to_csv("test_output_1.csv", index=False)